# Rightmove House Prices API — Examples

This notebook demonstrates how to use the API endpoints.

**Prerequisites:** Start the server with `uvicorn app.main:app --reload`

In [1]:
import json
import requests

BASE = "http://localhost:8000"

def pp(resp):
    """Pretty-print a response."""
    print(f"{resp.status_code} {resp.request.method} {resp.request.url}")
    try:
        print(json.dumps(resp.json(), indent=2))
    except Exception:
        print(resp.text[:500])

## 1. Basic Postcode Scrape

Fast path — extracts data from the listing page (single HTTP request).

In [2]:
resp = requests.post(f"{BASE}/scrape/postcode/E1W1AT", params={"max_properties": 10})
pp(resp)

200 POST http://localhost:8000/scrape/postcode/E1W1AT?max_properties=10
{
  "message": "Scraped 10 properties for postcode E1W1AT",
  "properties_scraped": 10
}


## 2. Multi-Page Scrape

Scrape 3 listing pages to get more properties.

In [ ]:
resp = requests.post(f"{BASE}/scrape/postcode/SW1A2AA", params={"pages": 3, "max_properties": 30})
pp(resp)

## 3. Single Property with Floorplan

Scrape a single property detail page and extract floorplan URLs.

In [ ]:
# First, get a property URL from a postcode scrape
props = requests.get(f"{BASE}/properties", params={"postcode": "E1W", "limit": 1}).json()
if props:
    url = props[0].get("url")
    print(f"Using property URL: {url}")
    if url:
        resp = requests.post(f"{BASE}/scrape/property", json={"url": url, "floorplan": True})
        pp(resp)
else:
    print("No properties found. Run cell 1 first.")

## 4. Postcode Scrape with Detail Pages and Floorplans

Slow path — visits individual detail pages for richer data.
`link_count=3` limits detail page visits. `floorplan=true` extracts floorplan URLs.

In [ ]:
resp = requests.post(
    f"{BASE}/scrape/postcode/E1W1AT",
    params={"link_count": 3, "floorplan": True},
)
pp(resp)

## 5. Query Properties with Filters

Search stored properties by postcode, type, and bedroom count.

In [ ]:
resp = requests.get(f"{BASE}/properties", params={
    "postcode": "E1W",
    "min_bedrooms": 2,
    "limit": 5,
})
pp(resp)

## 6. Property Detail with Sale History

Fetch a single property by ID to see its full sale history.

In [3]:
# Get the first property ID
props = requests.get(f"{BASE}/properties", params={"limit": 1}).json()
if props:
    pid = props[0]["id"]
    resp = requests.get(f"{BASE}/properties/{pid}")
    pp(resp)
else:
    print("No properties stored yet.")

200 GET http://localhost:8000/properties/10
{
  "id": 10,
  "address": "Flat 21, Ivory House, East Smithfield, London E1W 1AT",
  "postcode": "E1W 1AT",
  "property_type": "FLAT",
  "bedrooms": 2,
  "bathrooms": 2,
  "url": "https://www.rightmove.co.uk/house-prices/details/8a4c2bf1-687f-4cb2-aeb7-1f932dce12a0",
  "created_at": "2026-02-05T16:45:13.062717",
  "updated_at": "2026-02-05T16:45:13.062717",
  "extra_features": null,
  "sales": [
    {
      "id": 24,
      "date_sold": "10 Jun 2005",
      "price": "\u00a3661,500",
      "price_change_pct": "",
      "property_type": "",
      "tenure": "LEASEHOLD"
    },
    {
      "id": 23,
      "date_sold": "18 Dec 2017",
      "price": "\u00a3900,000",
      "price_change_pct": "",
      "property_type": "",
      "tenure": "LEASEHOLD"
    }
  ]
}


## 7. Postcode Summary

List all scraped postcodes with property counts.

In [ ]:
resp = requests.get(f"{BASE}/postcodes")
pp(resp)

## 8. Multi-Postcode Workflow

Scrape several postcodes, then query the combined results.

In [ ]:
postcodes = ["E1W1AT", "SE10DX", "N19AG"]
for pc in postcodes:
    resp = requests.post(f"{BASE}/scrape/postcode/{pc}", params={"max_properties": 10})
    data = resp.json()
    print(f"{pc}: {data.get('properties_scraped', 0)} properties")

print("\n--- Postcode summary ---")
resp = requests.get(f"{BASE}/postcodes")
for item in resp.json():
    print(f"  {item['postcode']}: {item['property_count']} properties")

## 9. Reading the Database Directly

You can also query the SQLite database directly, bypassing the API.
This is useful for data analysis, bulk exports, or integration with pandas.

In [1]:
import sqlite3
import json
import pandas as pd

DB_PATH = "rightmove.db"  # relative to where the server was started
conn = sqlite3.connect(DB_PATH)

# List all tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name", conn)
print("Tables:", tables["name"].tolist())

Tables: ['properties', 'sales']


### 9a. Browse all properties

In [2]:
pd.read_sql_query(
    "SELECT id, address, postcode, property_type, bedrooms, bathrooms FROM properties LIMIT 10",
    conn,
)

,id,address,postcode,property_type,bedrooms,bathrooms
0,1,"Flat 6, Ivory House, East Smithfield, London E...",E1W 1AT,FLAT,1,1
1,2,"Flat 23, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,1
2,3,"Flat 25, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,4,0
3,4,"Flat 30, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,2,1
4,5,"Flat 32, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,1
5,6,"Flat 29, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,4,4
6,7,"Flat 34, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,0
7,8,"Flat 15, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,0,0
8,9,"Flat 4, Ivory House, East Smithfield, London E...",E1W 1AT,FLAT,2,0
9,10,"Flat 21, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,2,2


### 9b. Sale history for a property

In [3]:
# Pick the first property and show its sales
first_id = pd.read_sql_query("SELECT id, address FROM properties LIMIT 1", conn)
if not first_id.empty:
    pid = int(first_id.iloc[0]["id"])
    print(f"Sales for: {first_id.iloc[0]['address']}\n")
    display(pd.read_sql_query(
        "SELECT date_sold, price, tenure, property_type FROM sales WHERE property_id = ? ORDER BY rowid",
        conn, params=(pid,),
    ))
else:
    print("No properties in the database yet.")

Sales for: Flat 15, Ivory House, East Smithfield, London E1W 1AT



,date_sold,price,tenure,property_type
0,30 Oct 2018,"£1,375,000",LEASEHOLD,
1,22 Dec 2005,"£670,000",LEASEHOLD,


### 9c. Join properties with their latest sale

In [4]:
pd.read_sql_query("""
    SELECT p.address, p.postcode, p.bedrooms, s.date_sold, s.price, s.tenure
    FROM properties p
    JOIN sales s ON s.property_id = p.id
    WHERE s.id = (
        SELECT s2.id FROM sales s2
        WHERE s2.property_id = p.id
        ORDER BY s2.rowid DESC LIMIT 1
    )
    ORDER BY p.postcode
    LIMIT 15
""", conn)

,address,postcode,bedrooms,date_sold,price,tenure
0,"Flat 6, Ivory House, East Smithfield, London E...",E1W 1AT,1,04 Nov 2005,"£397,000",Leasehold
1,"Flat 23, Ivory House, East Smithfield, London ...",E1W 1AT,1,6 Jul 2006,"£499,500",LEASEHOLD
2,"Flat 25, Ivory House, East Smithfield, London ...",E1W 1AT,4,2 Aug 2006,"£1,800,000",LEASEHOLD
3,"Flat 30, Ivory House, East Smithfield, London ...",E1W 1AT,2,1 Sep 2006,"£425,000",LEASEHOLD
4,"Flat 32, Ivory House, East Smithfield, London ...",E1W 1AT,1,30 Jun 2006,"£470,000",LEASEHOLD
5,"Flat 29, Ivory House, East Smithfield, London ...",E1W 1AT,4,20 Oct 2006,"£1,810,000",LEASEHOLD
6,"Flat 34, Ivory House, East Smithfield, London ...",E1W 1AT,1,22 Aug 2008,"£480,000",LEASEHOLD
7,"Flat 15, Ivory House, East Smithfield, London ...",E1W 1AT,0,22 Dec 2005,"£670,000",LEASEHOLD
8,"Flat 4, Ivory House, East Smithfield, London E...",E1W 1AT,2,6 May 2005,"£660,000",LEASEHOLD
9,"Flat 21, Ivory House, East Smithfield, London ...",E1W 1AT,2,10 Jun 2005,"£661,500",LEASEHOLD


### 9d. Aggregate stats by postcode

In [5]:
pd.read_sql_query("""
    SELECT
        p.postcode,
        COUNT(DISTINCT p.id) AS properties,
        COUNT(s.id) AS total_sales,
        ROUND(AVG(p.bedrooms), 1) AS avg_beds,
        GROUP_CONCAT(DISTINCT p.property_type) AS types
    FROM properties p
    LEFT JOIN sales s ON s.property_id = p.id
    WHERE p.postcode IS NOT NULL
    GROUP BY p.postcode
    ORDER BY properties DESC
""", conn)

,postcode,properties,total_sales,avg_beds,types
0,E1W 1AT,10,24,1.8,FLAT


### 9e. Properties with floorplan URLs

In [6]:
df_fp = pd.read_sql_query(
    "SELECT address, floorplan_urls FROM properties WHERE floorplan_urls IS NOT NULL LIMIT 10",
    conn,
)
if not df_fp.empty:
    # Parse JSON column into a proper list
    df_fp["floorplan_urls"] = df_fp["floorplan_urls"].apply(json.loads)
    display(df_fp)
else:
    print("No floorplan URLs stored yet. Run a scrape with floorplan=true first (cells 3 or 4).")

No floorplan URLs stored yet. Run a scrape with floorplan=true first (cells 3 or 4).


### 9f. Load into pandas DataFrame

In [7]:
df_props = pd.read_sql_query("SELECT * FROM properties", conn)
df_sales = pd.read_sql_query("SELECT * FROM sales", conn)

print(f"Properties: {len(df_props)} rows, Sales: {len(df_sales)} rows\n")
print("--- Properties ---")
display(df_props[["address", "postcode", "property_type", "bedrooms", "bathrooms"]].head())
print("\n--- Sales ---")
display(df_sales[["property_id", "date_sold", "price", "tenure"]].head())

Properties: 10 rows, Sales: 24 rows

--- Properties ---


,address,postcode,property_type,bedrooms,bathrooms
0,"Flat 6, Ivory House, East Smithfield, London E...",E1W 1AT,FLAT,1,1
1,"Flat 23, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,1
2,"Flat 25, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,4,0
3,"Flat 30, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,2,1
4,"Flat 32, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,1



--- Sales ---


,property_id,date_sold,price,tenure
0,1,7 Mar 2025,"£752,000",LEASEHOLD
1,1,4 Nov 2005,"£397,000",LEASEHOLD
2,2,15 Nov 2024,"£870,000",LEASEHOLD
3,2,6 Jul 2006,"£499,500",LEASEHOLD
4,3,1 Nov 2022,"£3,400,000",LEASEHOLD


### 9g. Export to CSV

In [8]:
df = pd.read_sql_query("""
    SELECT p.address, p.postcode, p.property_type, p.bedrooms, p.bathrooms,
           s.date_sold, s.price, s.tenure
    FROM properties p
    JOIN sales s ON s.property_id = p.id
    ORDER BY p.postcode, p.address, s.rowid
""", conn)

out_path = "rightmove_export.csv"
df.to_csv(out_path, index=False)
print(f"Exported {len(df)} rows to {out_path}")
df.head()

Exported 24 rows to rightmove_export.csv


,address,postcode,property_type,bedrooms,bathrooms,date_sold,price,tenure
0,"Flat 15, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,0,0,30 Oct 2018,"£1,375,000",LEASEHOLD
1,"Flat 15, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,0,0,22 Dec 2005,"£670,000",LEASEHOLD
2,"Flat 21, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,2,2,18 Dec 2017,"£900,000",LEASEHOLD
3,"Flat 21, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,2,2,10 Jun 2005,"£661,500",LEASEHOLD
4,"Flat 23, Ivory House, East Smithfield, London ...",E1W 1AT,FLAT,1,1,15 Nov 2024,"£870,000",LEASEHOLD


In [ ]:
conn.close()
print("Database connection closed.")

Database connection closed.


: 

In [7]:
import pandas as pd

pd.read_parquet('sales_data/enriched_properties.parquet ')

,address,postcode,property_type,bedrooms,bathrooms,extra_features,floorplan_urls,url,date_sold,date_sold_iso,...,swimming_pool,air_conditioning,solar_panels,loft,entrance_hall,white_goods,bay_window,distance_to_station,intercom,split_level
0,"100, Amity Grove, Raynes Park, London SW20 0LJ",SW20 0LJ,"Detached,Freehold",3,2,"[""Detached family home"", ""Three Double Bedroom...",None,https://www.rightmove.co.uk/house-prices/detai...,10 Jun 2013,2013-06-10,...,None,None,None,None,None,None,None,NaN,None,None
1,"100, Amity Grove, Raynes Park, London SW20 0LJ",SW20 0LJ,"Detached,Freehold",3,2,"[""Detached family home"", ""Three Double Bedroom...",None,https://www.rightmove.co.uk/house-prices/detai...,23 Feb 1996,1996-02-23,...,None,None,None,None,None,None,None,NaN,None,None
2,"100, Amity Grove, Raynes Park, London SW20 0LJ",SW20 0LJ,"Detached,Freehold",3,2,"[""Detached family home"", ""Three Double Bedroom...",None,https://www.rightmove.co.uk/house-prices/detai...,29 Aug 2024,2024-08-29,...,None,None,None,None,None,None,None,NaN,None,None
3,"100, Beverley Way, West Wimbledon, London SW20...",SW20 0AQ,"Semi-detached,Freehold",4,2,"[""CHAIN FREE"", ""Semi-Detached Mock Tudor Style...",None,https://www.rightmove.co.uk/house-prices/detai...,10 Aug 2012,2012-08-10,...,None,None,None,None,None,None,None,NaN,None,None
4,"100, Beverley Way, West Wimbledon, London SW20...",SW20 0AQ,"Semi-detached,Freehold",4,2,"[""CHAIN FREE"", ""Semi-Detached Mock Tudor Style...",None,https://www.rightmove.co.uk/house-prices/detai...,9 Mar 2021,2021-03-09,...,None,None,None,None,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,"Garden House, Cottenham Park Road, Wimbledon S...",SW20 0DR,"Flat,Leasehold",2,0,None,None,https://www.rightmove.co.uk/house-prices/detai...,12 Apr 2002,2002-04-12,...,None,None,None,None,None,None,None,NaN,None,None
2226,"Garden House, Cottenham Park Road, Wimbledon S...",SW20 0DR,"Flat,Leasehold",2,0,None,None,https://www.rightmove.co.uk/house-prices/detai...,29 Jan 1996,1996-01-29,...,None,None,None,None,None,None,None,NaN,None,None
2227,"Garden House, Cottenham Park Road, Wimbledon S...",SW20 0DR,"Flat,Leasehold",2,0,None,None,https://www.rightmove.co.uk/house-prices/detai...,31 Aug 1999,1999-08-31,...,None,None,None,None,None,None,None,NaN,None,None
2228,"Garden House, Cottenham Park Road, Wimbledon S...",SW20 0DR,"Flat,Leasehold",2,0,None,None,https://www.rightmove.co.uk/house-prices/detai...,5 Jun 1998,1998-06-05,...,None,None,None,None,None,None,None,NaN,None,None
